In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

from skimage.color import rgb2gray, gray2rgb

Using TensorFlow backend.


In [2]:
def load_data():
        color = []
        bw = []
        paths = []
        #get all files in this folder
        for r, d, f in os.walk(r"E:\DF\DFL2\workspace\data_dst"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))
        #for each file add normal resolution and small resolution to arrays
        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            placeholder = np.array(x)
            #placeholder = placeholder[...,:3]
            color.append(placeholder)
            bw.append(rgb2gray(placeholder))
            
        del paths    
       
        y_train = np.array(color)
        y_train = y_train.reshape(len(color),128,128,3)
        x_train = np.array(bw)
        x_train = x_train.reshape(len(bw),128,128,1)
        return y_train, x_train

In [3]:
class SRGAN():
    def __init__(self):
        
        # Shape of high resolution output image
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Shape of low resolution input image
        self.latent_dim = (128,128,1)

        #optimizer (learning rate and beta values)
        optimizer = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=self.latent_dim)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Conv2D(64, input_shape=self.latent_dim, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=self.latent_dim)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        Y_train, X_train = load_data()

        # Rescale to be between 0 & 1
        X_train = X_train / 255
        Y_train = Y_train / 255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Placeholder for loss function values
        g_loss_epochs = np.zeros((epochs, 1))
        d_loss_epochs = np.zeros((epochs, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, Y_train.shape[0], batch_size)
            imgs = Y_train[idx]

            # Generate super resolution images from the random batch of images
            gen_imgs = self.generator.predict(X_train[idx])

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(X_train[idx], valid)
            
            #save loss history
            g_loss_epochs[epoch] = g_loss
            d_loss_epochs[epoch] = d_loss[0]

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, X_train, idx)
                
        return g_loss_epochs, d_loss_epochs

    def save_imgs(self, epoch, X_train, idx):
        r, c = 3, 3
        # Select 9 random images
        index = np.random.randint(0, X_train.shape[0], 9)
        images = X_train[idx]
        # Super resolution the images
        gen_imgs = self.generator.predict(images)
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)
        # Plot each image
        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        # save model to .h5 file
        self.generator.save(r"C:\Users\Vee\Desktop\python\GAN\models\generator" + str(epoch) + ".h5")

In [4]:
gan = SRGAN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 64)       

In [5]:
g_loss, d_loss = gan.train(epochs=15001, batch_size=9, save_interval=100)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0

0 [D loss: 1.069446, acc.: 27.78%] [G loss: 0.215920]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.284826, acc.: 50.00%] [G loss: 0.346180]
2 [D loss: 0.882214, acc.: 50.00%] [G loss: 0.706209]
3 [D loss: 0.954131, acc.: 33.33%] [G loss: 1.434411]
4 [D loss: 0.904467, acc.: 50.00%] [G loss: 0.942101]
5 [D loss: 0.505538, acc.: 66.67%] [G loss: 0.898067]
6 [D loss: 0.763928, acc.: 55.56%] [G loss: 0.930077]
7 [D loss: 0.806513, acc.: 44.44%] [G loss: 0.645014]
8 [D loss: 0.898600, acc.: 33.33%] [G loss: 0.454336]
9 [D loss: 0.359839, acc.: 94.44%] [G loss: 0.735464]
10 [D loss: 0.449001, acc.: 83.33%] [G loss: 0.625359]
11 [D loss: 0.413328, acc.: 83.33%] [G loss: 0.267748]
12 [D loss: 0.495762, acc.: 77.78%] [G loss: 0.509484]
13 [D loss: 0.529984, acc.: 72.22%] [G loss: 0.408562]
14 [D loss: 0.424786, acc.: 77.78%] [G loss: 0.225326]
15 [D loss: 0.487220, acc.: 77.78%] [G loss: 0.231468]
16 [D loss: 0.208330, acc.: 94.44%] [G loss: 0.271630]
17 [D loss: 0.428959, acc.: 83.33%] [G loss: 0.099438]
18 [D loss: 0.377505, acc.: 83.33%] [G loss: 0.167428]
19 [D loss: 0.44861

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

14800 [D loss: 0.001140, acc.: 100.00%] [G loss: 1.884206]
14801 [D loss: 0.000664, acc.: 100.00%] [G loss: 0.240650]
14802 [D loss: 0.000054, acc.: 100.00%] [G loss: 0.124878]
14803 [D loss: 0.000712, acc.: 100.00%] [G loss: 0.026760]
14804 [D loss: 0.568702, acc.: 94.44%] [G loss: 1.977638]
14805 [D loss: 0.000721, acc.: 100.00%] [G loss: 0.511511]
14806 [D loss: 0.054064, acc.: 100.00%] [G loss: 0.098611]
14807 [D loss: 0.000611, acc.: 100.00%] [G loss: 0.708326]
14808 [D loss: 0.000018, acc.: 100.00%] [G loss: 0.658010]
14809 [D loss: 0.000022, acc.: 100.00%] [G loss: 0.065938]
14810 [D loss: 0.000141, acc.: 100.00%] [G loss: 0.038177]
14811 [D loss: 0.018889, acc.: 100.00%] [G loss: 1.164136]
14812 [D loss: 0.000205, acc.: 100.00%] [G loss: 0.186562]
14813 [D loss: 0.000054, acc.: 100.00%] [G loss: 3.393307]
14814 [D loss: 0.000036, acc.: 100.00%] [G loss: 0.518639]
14815 [D loss: 0.000077, acc.: 100.00%] [G loss: 0.167649]
14816 [D loss: 0.040181, acc.: 94.44%] [G loss: 0.022605]

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


14900 [D loss: 0.000047, acc.: 100.00%] [G loss: 0.080814]
14901 [D loss: 0.004454, acc.: 100.00%] [G loss: 0.507943]
14902 [D loss: 0.170582, acc.: 94.44%] [G loss: 0.159916]
14903 [D loss: 0.000083, acc.: 100.00%] [G loss: 0.432150]
14904 [D loss: 0.003442, acc.: 100.00%] [G loss: 1.581329]
14905 [D loss: 0.121323, acc.: 94.44%] [G loss: 2.187139]
14906 [D loss: 0.646003, acc.: 83.33%] [G loss: 2.528031]
14907 [D loss: 0.000066, acc.: 100.00%] [G loss: 0.248860]
14908 [D loss: 0.000027, acc.: 100.00%] [G loss: 0.040710]
14909 [D loss: 0.000315, acc.: 100.00%] [G loss: 0.224317]
14910 [D loss: 0.051785, acc.: 94.44%] [G loss: 1.917587]
14911 [D loss: 0.000168, acc.: 100.00%] [G loss: 1.230187]
14912 [D loss: 0.009448, acc.: 100.00%] [G loss: 0.350806]
14913 [D loss: 0.000302, acc.: 100.00%] [G loss: 0.565662]
14914 [D loss: 0.000026, acc.: 100.00%] [G loss: 0.428947]
14915 [D loss: 0.000153, acc.: 100.00%] [G loss: 0.595398]
14916 [D loss: 0.000061, acc.: 100.00%] [G loss: 1.112937]
1

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


15000 [D loss: 0.000001, acc.: 100.00%] [G loss: 0.181766]
